In [1]:
from typing import Tuple, List
from math import log

rates = [
    [1, 0.23, 0.25, 16.43, 18.21, 4.94],
    [4.34, 1, 1.11, 71.40, 79.09, 21.44],
    [3.93, 0.90, 1, 64.52, 71.48, 19.37],
    [0.061, 0.014, 0.015, 1, 1.11, 0.30],
    [0.055, 0.013, 0.014, 0.90, 1, 0.27],
    [0.20, 0.047, 0.052, 3.33, 3.69, 1],
]

currencies = ('PLN', 'EUR', 'USD', 'RUB', 'INR', 'MXN')


def negate_logarithm_convertor(graph: Tuple[Tuple[float]]) -> List[List[float]]:
    ''' log of each rate in graph and negate it'''
    result = [[-log(edge) for edge in row] for row in graph]
    return result


def arbitrage(currency_tuple: tuple, rates_matrix: Tuple[Tuple[float, ...]]):
    ''' Calculates arbitrage situations and prints out the details of this calculations'''

    trans_graph = negate_logarithm_convertor(rates_matrix)

    # Pick any source vertex -- we can run Bellman-Ford from any vertex and get the right result

    source = 0
    n = len(trans_graph)
    min_dist = [float('inf')] * n

    pre = [-1] * n
    
    min_dist[source] = source

    # 'Relax edges |V-1| times'
    for _ in range(n-1):
        for source_curr in range(n):
            for dest_curr in range(n):
                if min_dist[dest_curr] > min_dist[source_curr] + trans_graph[source_curr][dest_curr]:
                    min_dist[dest_curr] = min_dist[source_curr] + trans_graph[source_curr][dest_curr]
                    pre[dest_curr] = source_curr

    # if we can still relax edges, then we have a negative cycle
    for source_curr in range(n):
        for dest_curr in range(n):
            if min_dist[dest_curr] > min_dist[source_curr] + trans_graph[source_curr][dest_curr]:
                # negative cycle exists, and use the predecessor chain to print the cycle
                print_cycle = [dest_curr, source_curr]
                # Start from the source and go backwards until you see the source vertex again or any vertex that already exists in print_cycle array
                while pre[source_curr] not in  print_cycle:
                    print_cycle.append(pre[source_curr])
                    source_curr = pre[source_curr]
                print_cycle.append(pre[source_curr])
                print("Arbitrage Opportunity: \n")
                print(" --> ".join([currencies[p] for p in print_cycle[::-1]]))


if __name__ == "__main__":
    arbitrage(currencies, rates)

# Time Complexity: O(N^3)
# Space Complexity: O(N^2)

Arbitrage Opportunity: 

RUB --> INR --> PLN --> RUB
Arbitrage Opportunity: 

USD --> MXN --> USD --> RUB --> INR --> EUR --> PLN
Arbitrage Opportunity: 

USD --> MXN --> USD --> PLN


In [2]:
import ccxt
import pandas as pd
import time

pd.set_option('expand_frame_repr', False)

def main():
    """
        主函数
    """
    # 初始化交易所
    binance_exchange = ccxt.binance({
        'timeout': 15000,
        'enableRateLimit': True,
        'proxies': {'https': "http://127.0.0.1:8001", 'http': "http://127.0.0.1:8001"}
    #上面的Proxy改成你的软件上“Copy HTTP Proxy Shell Export Line”的链接
    })

    # 加载行情
    markets = binance_exchange.load_markets()

    # == Step.1 选择两个交易市场 A, B
    market_a = 'BTC'
    market_b = 'ETH'
    # == Step.1 END =================

    # == Step.2 找到所有同时以 A 和 B 都作为计价的货币
    # 市场内的交易对
    symbols = list(markets.keys())

    # 存放到DataFrame中
    symbols_df = pd.DataFrame(data=symbols, columns=['symbol'])

    # 分割字符串得到 基础货币/计价货币
    base_quote_df = symbols_df['symbol'].str.split(pat='/', expand=True)
    base_quote_df.columns = ['base', 'quote']

    # 过滤得到以 A, B 计价的计价货币
    base_a_list = base_quote_df[base_quote_df['quote'] == market_a]['base'].values.tolist()
    base_b_list = base_quote_df[base_quote_df['quote'] == market_b]['base'].values.tolist()

    # 获取相同的基础货币列表
    common_base_list = list(set(base_a_list).intersection(set(base_b_list)))
    print('{}和{}共有{}个相同的计价货币'.format(market_a, market_b, len(common_base_list)))
    # == Step.2 END =================

    # == Step.3 执行套利步骤

    # 结果保存到DataFrame中，注意，这里的Profit为千分位 ‰（因为利润一般都比较小）
    columns = ['Market A',
               'Market B',
               'Market C',
               'P1',
               'P2',
               'P3',
               'Profit']

    results_df = pd.DataFrame(columns=columns)

    # 获取前一分钟的close价格
    last_min = binance_exchange.milliseconds() - 60 * 1000  # 前一分钟

    for base_coin in common_base_list:
        market_c = base_coin
        market_a2b_symbol = '{}/{}'.format(market_b, market_a)
        market_b2c_symbol = '{}/{}'.format(market_c, market_b)
        market_a2c_symbol = '{}/{}'.format(market_c, market_a)

        # 获取行情前一分钟的K线数据
        market_a2b_kline = binance_exchange.fetch_ohlcv(market_a2b_symbol, since=last_min, limit=1, timeframe='1m')
        market_b2c_kline = binance_exchange.fetch_ohlcv(market_b2c_symbol, since=last_min, limit=1, timeframe='1m')
        market_a2c_kline = binance_exchange.fetch_ohlcv(market_a2c_symbol, since=last_min, limit=1, timeframe='1m')

        if len(market_a2b_kline) == 0 or len(market_b2c_kline) == 0 or market_a2c_kline == 0:
            continue

        try:

            # 获取行情前一分钟的交易对价格
            p1 = market_a2b_kline[0][4]
            p2 = market_b2c_kline[0][4]
            p3 = market_a2c_kline[0][4]
        except IndexError:
            continue

        # 价差
        profit = (p3 / (p1 * p2) - 1) * 1000

        results_df = results_df.append({
            'Market A': market_a,
            'Market B': market_b,
            'Market C': market_c,
            'P1': p1,
            'P2': p2,
            'P3': p3,
            'Profit': profit
        }, ignore_index=True)

        # 显示信息
        print(results_df.tail(1))

        # 防止超过rate limit
        time.sleep(binance_exchange.rateLimit / 1000)
    # == Step.3 END =================

    # 按照收益由高到低排序
    # @see https://www.jianshu.com/p/d12af2b287b6
    results_df = results_df.sort_values(by="Profit", ascending=False)
    # print(new_sort)

    results_df.to_csv('./tri_arbitrage_results.csv', index=None)

if __name__ == '__main__':
    main()

BTC和ETH共有159个相同的计价货币
  Market A Market B Market C        P1       P2        P3    Profit
0      BTC      ETH    THETA  0.062219  0.00227  0.000141 -0.439025
  Market A Market B Market C        P1        P2        P3    Profit
1      BTC      ETH      ICX  0.062212  0.000388  0.000024  4.215052
  Market A Market B Market C       P1        P2        P3    Profit
2      BTC      ETH      BNT  0.06221  0.001415  0.000088  2.986011
  Market A Market B Market C        P1        P2        P3    Profit
3      BTC      ETH      ADX  0.062225  0.000172  0.000011 -0.377432
  Market A Market B Market C       P1        P2       P3    Profit
4      BTC      ETH      GLM  0.06223  0.000153  0.00001 -2.081687
  Market A Market B Market C        P1        P2        P3    Profit
5      BTC      ETH      XLM  0.062231  0.000126  0.000008  0.971139
  Market A Market B Market C        P1        P2        P3    Profit
6      BTC      ETH      VET  0.062235  0.000034  0.000002  2.467933
  Market A Market B M